In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
np.set_printoptions(threshold=np.nan)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
arr = np.array([1,2,3,4,5,6,7,8,9,10])

In [4]:
print(arr.mean())
print(arr.sum())

5.5
55


In [25]:
df = pd.read_csv("data/application_train.csv.zip")
df_test = pd.read_csv("data/application_test.csv.zip")

In [7]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# How many classes
df["TARGET"].nunique()

2

In [14]:
# Distribution of those classes
df["TARGET"].value_counts(dropna=False)

0    282686
1     24825
Name: TARGET, dtype: int64

In [27]:
dtypes = df.dtypes
dtypes = dtypes[dtypes!='object']
features = list(set(dtypes.index)-set(['TARGET']))
len(features)

105

In [29]:
X = df[features]
y = df['TARGET']

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
model = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=16, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=16, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=40,
       reg_lambda=40, scale_pos_weight=4, seed=None, silent=True,
       subsample=1)

In [37]:
from sklearn.metrics import roc_auc_score

y_train_predicted = model.predict_proba(X_train)[:,1]
y_test_predicted = model.predict_proba(X_test)[:,1]

print("Train AUC %.4f"%roc_auc_score(y_train,y_train_predicted))
print("Test AUC %.4f"%roc_auc_score(y_test,y_test_predicted))

Train AUC 0.7746
Test AUC 0.7540


In [38]:
def generate_results(model,df_test,features,id_col,target,file):
    dft = df_test[features]
    results = df_test[[id_col]]
    results[target] = model.predict_proba(dft)[:,1]
    results.to_csv(file,index=False,columns=results.columns)
    
generate_results(model,df_test,features,"SK_ID_CURR","TARGET","results/results.csv")

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
!pwd

/home/ubuntu/data-science/kaggle/home-credit-default-risk


In [39]:
!head -n10 results/results.csv

SK_ID_CURR,TARGET
100001,0.16432536
100005,0.26694274
100013,0.07495012
100028,0.108937845
100038,0.4518704
100042,0.12994887
100057,0.07814581
100065,0.13142742
100066,0.07274422


In [41]:
%cd  results

!kaggle  competitions  submit -c home-credit-default-risk -f results.csv -m "Xgb baseline with numeric cols"
%cd ..

/home/ubuntu/data-science/kaggle/home-credit-default-risk/results
Successfully submitted to Home Credit Default Risk/home/ubuntu/data-science/kaggle/home-credit-default-risk


In [42]:
!kaggle competitions submissions -c home-credit-default-risk -q

fileName       date                 description                     status    publicScore  privateScore  
-------------  -------------------  ------------------------------  --------  -----------  ------------  
results.csv    2018-05-27 11:05:15  Xgb baseline with numeric cols  complete  0.734        None          
results-4.csv  2018-05-27 06:39:32  Xgb base                        complete  0.741        None          
results-4.csv  2018-05-27 06:22:13  Xgb base                        complete  0.743        None          
results-3.csv  2018-05-26 20:10:22  Xgb base                        complete  0.742        None          
results-2.csv  2018-05-26 20:09:25  Xgb base                        complete  0.743        None          
results.csv    2018-05-26 19:50:53  Xgb base                        complete  0.744        None          
